# 수면장애 데이터 전처리

- 2021년도 데이터만 파싱
- 시군구2 : 경기/인천 '구' 단위 → '시' 단위 통합

In [1]:
import pandas as pd

In [22]:
# 데이터 불러오기
sleep_df = pd.read_csv('./data/raw/국민건강보험공단_수면장애 관련 주상병별 진료정보_20231231.csv', encoding='cp949')
sleep_df.head()

,주상병코드,주상병명,진료년도,시도,시군구,진료인원(명),진료비(천원),급여비(천원)
0,G47,수면장애,2021,서울특별시,종로구,2691,672059,472058
1,G47,수면장애,2021,서울특별시,중구,2208,561303,407649
2,G47,수면장애,2021,서울특별시,용산구,4640,1217122,869536
3,G47,수면장애,2021,서울특별시,성동구,5612,1314752,948009
4,G47,수면장애,2021,서울특별시,광진구,5594,1407419,1002798


In [23]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2367 entries, 0 to 2366
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   주상병코드    2367 non-null   object
 1   주상병명     2367 non-null   object
 2   진료년도     2367 non-null   int64 
 3   시도       2367 non-null   object
 4   시군구      2367 non-null   object
 5   진료인원(명)  2366 non-null   object
 6   진료비(천원)  2367 non-null   int64 
 7   급여비(천원)  2367 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 148.1+ KB


In [24]:
# 2021년, '비기질성 수면장애'만 필터링
filter = (sleep_df['진료년도'] == 2021) & (sleep_df['주상병명'] == '비기질성 수면장애') & (sleep_df['시도'].isin(['서울특별시', '경기도', '인천광역시']))
sleep_2021 = sleep_df[filter]

In [ ]:
# 필요없는 column 삭제
sleep_2021.drop(columns=['주상병코드','진료비(천원)','급여비(천원)'], inplace =True)

In [27]:
sleep_2021.columns

Index(['주상병명', '진료년도', '시도', '시군구', '진료인원(명)'], dtype='object')

경기/인천 '구' 단위 → '시' 단위 통합

In [ ]:
sleep_2021['시군구2'] = sleep_2021['시군구']

In [29]:
def replace_values_with_multiple_substrings(sleep_2021, column, replacements):
  for old_value, new_value in replacements.items():
        sleep_2021.loc[sleep_2021[column].str.contains(old_value, na=False), column] = new_value
  return sleep_2021

replacements = {
    '수원시': '수원시',
    '성남시': '성남시',
    '안양시': '안양시',
    '부천시': '부천시',
    '안산시': '안산시',
    '고양시': '고양시',
    '용인시': '용인시',
}

updated_sleep_2021 = replace_values_with_multiple_substrings(sleep_2021, '시군구2', replacements)

데이터 저장

In [ ]:
import os
updated_sleep_2021.to_csv('.data/수면장애 관련 주상병별 진료정보_2020.csv', index=False)